In [2]:
# imports
import os
from skimage.measure import label,regionprops
from skimage.morphology import binary_opening, erosion, dilation, ball
from skimage.segmentation import watershed
from scipy.ndimage import median_filter as median
from skimage.filters import gaussian
import numpy as np
import nrrd
from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv 

In [3]:
# lists of image/mask files
img_path = './data/images'
msk_path = './data/masks'

all_images = sorted([os.path.join(img_path,f) for f in os.listdir(img_path)])
all_masks = sorted([os.path.join(msk_path,f) for f in os.listdir(msk_path)])

In [4]:
# first pass of segmentation pipeline
def segPipeline(imgList,mskList):
    
    # initialize lists for segmentation accuracy
    all_IoU = []
    all_backIoU = []
    num_lesions_per_patient = []
    
    for i in range(len(imgList)):
        
        # read image and corresponding mask
        im_V,img_d = nrrd.read(imgList[i])
        msk_V,msk_d = nrrd.read(mskList[i])
        
        # if the image and mask do not have the same size, do not process
        if not (img_d['sizes'] == msk_d['sizes']).all():
            print('ERROR, {}: image and mask must be of the same size'.format(imgList[i]))
            continue
            
        # label the mask into connected regions
        lesion_labels,num_lesions = label(msk_V,return_num=True)
        lesion_props = regionprops(lesion_labels)
        
        num_lesions_per_patient.append(num_lesions)
        
        # preprocess the original image:
        #        (i)   filter to minimize noise
        #        (ii)  obtain the morphological gradient
#         denoised = median(im_V,3)
        denoised = gaussian(im_V)
        gradient_image = dilation(denoised,ball(2)) - erosion(denoised,ball(2))
        
        # work towards a whole-image segmentation
        #        (i)   threshold the morphological gradient
        #        (ii)  label the threshold image
        #        (iii) apply watershed to threshold image using labels as markers
        threshold_gradient = gradient_image < 0.01 
        markers = label(threshold_gradient)
        labels = watershed(gradient_image, markers)
        
        # initialize whole watershed mask
        water_whole = np.zeros((msk_V.shape[0],msk_V.shape[1],msk_V.shape[2]),dtype=bool)
        
        for obj in lesion_props:
            
            # isolate the centroid of the nodule -- this will be our lesion identifier for automation
            centroid = [round(i) for i in obj.centroid] 
            
            # the segmentation
            water_mask = labels == labels[centroid[0],centroid[1],centroid[2]]
            
            # reconstruct the mask for each lesion using regionprops attributes
            msk_image = np.zeros((msk_V.shape[0],msk_V.shape[1],msk_V.shape[2]),dtype=bool)
            for coord in obj.coords:
                msk_image[coord[0],coord[1],coord[2]] = True
            
            # calculate the lesion-specific IoU
            all_IoU.append(np.count_nonzero(np.logical_and(water_mask,msk_image))/np.count_nonzero(np.logical_or(water_mask,msk_image)))

            # update the whole watershed mask
            water_whole = np.logical_or(water_whole,water_mask)

        
        # calculate the patient-specific IoU
        all_backIoU.append(np.count_nonzero(np.logical_and(~water_whole,~msk_V))/np.count_nonzero(np.logical_or(~water_whole,~msk_V)))

        
    return all_IoU, all_backIoU, num_lesions_per_patient
        

# run the pipeline
iou, bg_iou, num_l = segPipeline(all_images,all_masks)

In [5]:
iou

[0.7816774658027358,
 0.0002862477996909721,
 9.586470162882943e-06,
 0.4602272727272727,
 9.336707721857641e-05,
 0.6838477674557987,
 1.662195398993042e-06,
 1.2688080230020797e-05,
 1.6682511136163597e-05,
 3.266012743558788e-05,
 1.9982326400849464e-06,
 3.573762221690385e-06,
 1.4948317249866234e-05,
 6.26369077565089e-06,
 8.780872823007417e-05,
 0.6782317717506164,
 4.2488094304874596e-07,
 3.08739840790374e-06,
 2.62846080672886e-06,
 1.7106852963685989e-06,
 1.7569200341082907e-06,
 1.917439451356658e-06,
 3.4553862300901343e-06,
 5.792265934833173e-07]

In [7]:
bg_iou

[0.9998717630716195,
 0.2722985717071884,
 0.25808026450020927,
 0.30484152829498673,
 0.8779310068689791,
 0.25361013770999763,
 0.628368824925916,
 0.31253992525258456,
 0.3554127514362335,
 0.310507075450911]

In [6]:
# testing -- there are 4 lesions in this patient
# im_V,img_d = nrrd.read(all_images[2])
# msk_V,msk_d = nrrd.read(all_masks[2])

# lesion_labels,num_lesions = label(msk_V,return_num=True)
# lesion_props = regionprops(lesion_labels)

In [7]:
# further testing -- build mask of 1st lesion using regionprops information
# msk_image = np.zeros((msk_V.shape[0],msk_V.shape[1],msk_V.shape[2]),dtype=bool)

# for coord in lesion_props[0].coords:
#     msk_image[coord[0],coord[1],coord[2]] = True
    
# isv(msk_image)

In [10]:
im_V,img_d = nrrd.read(all_images[1])
msk_V,msk_d = nrrd.read(all_masks[1])

lesion_labels,num_lesions = label(msk_V,return_num=True)
lesion_props = regionprops(lesion_labels)

# further testing -- build mask of 1st lesion using regionprops information
# msk_image = np.zeros((msk_V.shape[0],msk_V.shape[1],msk_V.shape[2]),dtype=bool)

# for coord in lesion_props[0].coords:
#     msk_image[coord[0],coord[1],coord[2]] = True
print(lesion_props[0].centroid)  
isv(im_V)

(359.9256946393489, 344.7133735616054, 183.6581532416503)


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…